## Heat equation in 4 dimensions

Comparing PINN solution to exact solution.

In [2]:
import torch
from model_architecture import PINN
pi = 3.1415926535898


def exact_solution(x_list, t):
    return torch.sin(pi*x_list[0])*torch.sin(2*pi*x_list[1])*torch.sin(pi*x_list[2])*torch.sin(pi*x_list[3])*torch.exp(-7*alpha*pi**2*t)

alpha = 0.05  # Thermal diffusivity
spatial_dim = 4

# Load PINN model
model = PINN(input_dim=spatial_dim+1)
model.load_state_dict(torch.load('weights1.pth'))

model.eval()
x1 = torch.linspace(0, 1, 20)
x2 = torch.linspace(0, 1, 20)
x3 = torch.linspace(0, 1, 20)
x4 = torch.linspace(0, 1, 20)
times = torch.tensor([0.0, 0.5, 1.0])

X1, X2, X3, X4 = torch.meshgrid(x1, x2, x3, x4, indexing='ij')

for i, t in enumerate(times):
    T = t * torch.ones_like(X1)
    with torch.no_grad():
        U = model([X1.reshape(-1, 1), X2.reshape(-1, 1), X3.reshape(-1, 1), X4.reshape(-1, 1)], T.reshape(-1, 1)).reshape(X1.shape)

    U_exact = exact_solution([X1, X2, X3, X4], T)
    
    abs_diff = torch.abs(U - U_exact)
    max_exact = torch.max(torch.abs(U_exact))
    diff_ratio = abs_diff / max_exact

    print('\nt = ', t.item())
    print('Max diff ratio:', torch.max(diff_ratio).item())
    print('Mean diff ratio:', torch.mean(diff_ratio).item())
    print('Median diff ratio:', torch.median(diff_ratio).item())


t =  0.0
Max diff ratio: 0.04120060056447983
Mean diff ratio: 0.0032018660567700863
Median diff ratio: 0.001976216211915016

t =  0.5
Max diff ratio: 0.04313846677541733
Mean diff ratio: 0.004334792960435152
Median diff ratio: 0.0013433638960123062

t =  1.0
Max diff ratio: 0.23114466667175293
Mean diff ratio: 0.025382166728377342
Median diff ratio: 0.008866611868143082
